This notebook builds all supplementary tables required for regenerating figures. A single, multi-tabbed excel file will be output.

In [ ]:
import pandas as pd
import altair as alt
from pathlib import Path
import numpy as np
import re

In [ ]:
#SGE Score Files
del_file = '../Data/supp_table_inputs/20251002_BARD1delscores.tsv' #BARD1 3bp deletion scores
snv_file = '../Data/supp_table_inputs/20251002_BARD1snvscores.tsv' #BARD1 SNV scores
thresholds = '../Data/supp_table_inputs/20251002_BARD1modelparams.tsv' #SGE pipeline output thresholds

#ClinVar Files
clinvar_snvs_file = '../Data/supp_table_inputs/20250912_BARD1_ClinVarSNVs_1StarPlus.txt' #ClinVar SNVs, at least 1 star review status, accessed 2025/09/12
clinvar_dels_file = '../Data/supp_table_inputs/20250912_BARD1_ClinVarDels_1StarPlus.txt' #ClinVar Deletions, at least 1 star review status, accessed 2025/09/12

#Allele Frequency Files
gnom_path = '../Data/supp_table_inputs/20240905_BARD1_gnomADv4.1.0_SNVs.xlsx' #gnomAD v4.1.0 SNVs, accessed 2024/09/05
reg_path = '../Data/supp_table_inputs/20240802_BARD1_Regeneron_MAF.xlsx' #Regeneron Million Exome allele frequency data. Accessed 2024/08/02

#Normalized Depth Files
read_depth_path = '../Data/supp_table_inputs/depth_data/depth_files'
gsp_input_file = '../Data/supp_table_inputs/depth_data/deletion_inputs.xlsx'
target_coords = '../Data/supp_table_inputs/20250415_BARD1_filter_entry.xlsx'
cut_sites = '../Data/supp_table_inputs/20241217_BARD1_sgRNA_cutsites.xlsx'

#Misc. Annotations
snv_counts = '../Data/supp_table_inputs/20250825_BARD1counts.tsv' #Counts for assayed SNVs
vep_predictions = '../Data/supp_table_inputs/20251002_BARD1snvs_VEP.xlsx' #VEP annotated file
mutpred_prediction = '../Data/supp_table_inputs/20251006_BARD1_MutPred2.xlsx' #MutPred2 scores
atg_scores = '../Data/supp_table_inputs/ATG_lib_data/20250409_BARD1_X1A_ATG_scored.xlsx' #ATG score file
rna_scores = '../Data/supp_table_inputs/20250922_BARD1RNAscores.tsv' #RNA scores
phylop = '../Data/supp_table_inputs/20251008_PhyloP.xlsx' #PhyloP scores
edit_rate = '../Data/supp_table_inputs/20250926_BARD1.editrates.sorted.tsv' #Editing rates for useable reads file
orthogonal_assays = '../Data/supp_table_inputs/20241016_Orthogonal_BARD1_FunctionalAssays.xlsx' #Curated orthogonal BARD1 assays
alt.data_transformers.disable_max_rows()

Functions for initial read in of SGE data

In [ ]:
def get_thresholds(snv, thresholds): #Gets SGE thresholds
    df = pd.read_csv(thresholds, sep = '\t')

    
    thresholds = [df['thresh_abnormal'][0], df['thresh_normal'][0]]


    df = pd.read_csv(snv, sep = '\t')
    #Some quick processing of SNV scores
    df.loc[df['score'] >= 0, 'functional_consequence'] = 'functionally_normal' #Ensures that variants above our upper threshold (which is less than 0) will be assigned a functionally normal class
    df['var_type'] = 'snv' #Sets variant type column to SNV
    df['pos'] = df['pos'].astype(int)
    df['start'] = df['pos']
    df['end'] = df['pos']
    df['pos_id'] = df['pos'].astype(str) + ':' + df['alt']
    
    return df, thresholds

In [ ]:
def class_dels(dels, thresholds):
    dels = pd.read_csv(dels, sep = '\t') #Reads deletions
    
    dels['var_type'] = '3bp_del'
    dels['start'] = dels['pos'] + 1
    dels['end'] = dels['pos'] + 3
    dels['pos_id'] = dels['start'].astype(str) + "-" + dels['end'].astype(str)
    
    dels = dels.astype({'pos': int,
                       'start': int,
                       'end': int})
    
    return dels

Function used to process and merge RNA data

In [ ]:
def process_rna(rna):
    df = pd.read_csv(rna, sep = '\t')
    df = df.loc[~np.isinf(df['RNA_DNA_log2'])]


    df = df[['pos_id', 'RNA_DNA_log2', 'exon', 'consequence']]

    df = df.rename(columns = {'RNA_DNA_log2': 'RNAscore'})

    df = df.groupby('pos_id').agg({
        'RNAscore': 'mean',
        'consequence': 'first',
        'exon': 'first',
    }).reset_index()


    class_df = df.loc[df['consequence'].isin(['stop_gained'])]
    class_df = class_df.copy()

    class_df = class_df.loc[~class_df['exon'].isin(['BARD1_X1', 'BARD1_X2', 'BARD1_X11'])]
    percentiles = class_df['RNAscore'].quantile(0.975)

    class_df = class_df.loc[class_df['RNAscore'] <= percentiles]
    

    mean_stop = class_df['RNAscore'].mean()
  
    mean_std = class_df['RNAscore'].std()

    lwrthresh = mean_stop  +  mean_std
 

    df['RNA_consequence'] = 'normal'
    df.loc[df['RNAscore'] <= lwrthresh, 'RNA_consequence'] = 'low'

    df = df[['pos_id', 'RNAscore', 'RNA_consequence']]

    return df, lwrthresh

Functions used to process and merge the ClinVar data

In [ ]:
def read_clinvar(snv_file, del_file): #Reads ClinVar data
    
    df = pd.read_csv(snv_file, delimiter='\t') #reads ClinVar SNV tabular .txt 
    df = df[['Name','Protein change','GRCh38Chromosome','GRCh38Location','Germline classification']] #pulls useful columns
    df = df.dropna(subset = ['GRCh38Location']) #Drops variants without genomic coordinate
    df.GRCh38Location = df.GRCh38Location.astype(int) #Sets coordinates to integer data type
    df['pos_id'] = None #preps for next function


    del_df = pd.read_csv(del_file, sep = '\t') #Reads ClinVar deletions
    del_df = del_df.loc[del_df['GRCh38Location'].str.contains('-')] #Splits coordinates
    del_df['start'] = del_df['GRCh38Location'].transform(lambda x: x.split(' - ')[0]) #Gets deletion start coordinate
    del_df['end'] = del_df['GRCh38Location'].transform(lambda x: x.split(' - ')[1]) #Gets deletion end coordinate

    #Sets coordinate data types to integer
    del_df['start'] = del_df['start'].astype(int) 
    del_df['end'] = del_df['end'].astype(int)

    del_df['del_length'] = del_df['end'] - del_df['start'] #Calculates deletion length 

    del_df = del_df.loc[del_df['del_length'].isin([2])] #Pulls out 3bp deletions
    del_df['pos_id'] = del_df['start'].astype(str) + '-' + del_df['end'].astype(str) #Sets base change column to coordinate spanned by deletion
    del_df = del_df[['pos_id', 'Germline classification']] #Pulls out necessary columns


    return df, del_df

In [ ]:
def get_pair(base): #ClinVar gives base changes on negative sense strand, SGE pos_id on positive sense
    if base == 'A':
        return 'T'
    elif base == 'T':
        return 'A'
    elif base == 'C':
        return 'G'
    else:
        return 'C'

In [ ]:
def get_base_changes(df): #Creates pos_id column in format of SGE datafile for ClinVar data    
    k = 0
    while k < len(df):
        var = df['Name'][k]
        coord = str(df['GRCh38Location'][k])
        k += 1
        i = 0
        j = 3
        while j < (len(var) + 1):
            test_str = var[i:j]
            j += 1
            i += 1
            sense_base = get_pair(test_str[2])
            if test_str[1] == '>':
                change = coord + ":" + sense_base
                df.loc[df['Name'] == var, 'pos_id'] = change
                
    df = df[['pos_id', 'Germline classification']]
    return df

Functions used to merge allele frequency data from gnomAD and Regeneron Million Exomes 

In [ ]:
def read_gnomAD(gnomAD_path): #Reads gnomAD file
    
    unfiltered = pd.read_excel(gnomAD_path) #Reads gnomAD file
    filtered = unfiltered[['gnomAD ID', 'Allele Frequency']] #Gets necessary columns 

    filtered = filtered.copy()
    filtered['pos_id'] = filtered['gnomAD ID'].transform(lambda x: x[2:11] + ':' + x[14]) #Adds pos_id column for merging

    filtered = filtered.rename(columns = {'Allele Frequency': 'gnomad_af'})
    filtered = filtered[['pos_id', 'gnomad_af']]
    return filtered

In [ ]:
def read_regeneron(reg_path): #Reads Regeneron data
    
    df = pd.read_excel(reg_path) #Reads data
    maf = df[['Variant','AAF']] #Pulls necessary columns
    maf = maf.copy()

    maf = maf.rename(columns = {'AAF': 'regeneron_maf', 'Variant': 'pos_id'}) #Renames columns to share column names with SGE data

    maf['pos_id'] = maf['pos_id'].transform(lambda x: x[2:12] + x[len(x) - 1: len(x) + 1]) #Remakes the pos_id column to match pos_id column from SGE data for merging
    
    return maf

Function to add SNV counts data

In [ ]:
def read_counts(counts):
    counts_df = pd.read_csv(counts, sep = '\t')

    counts_df['pos_id'] = counts_df['pos'].astype(str) + ':' + counts_df['alt']

    return counts_df

Fuction to add variant effect predictor data

In [ ]:
def read_vep(file, mutpred_file):
    vep_df = pd.read_excel(file)
    mutpred_df = pd.read_excel(mutpred_file)

    vep_df['pos'] = vep_df['Location'].transform(lambda x: x[-9:])
    vep_df['pos_id'] = vep_df['pos'] + ':' + vep_df['Allele']
    vep_df = vep_df.drop(columns = ['Location', 'Allele', 'pos'])
    
    vep_df['max_SpliceAI'] = vep_df[['SpliceAI_pred_DS_AG', 'SpliceAI_pred_DS_AL', 'SpliceAI_pred_DS_DG', 'SpliceAI_pred_DS_DL']].max(axis = 1)
    vep_df = vep_df.rename(columns = {'SpliceAI_pred_DS_AG': 'SpliceAI_AG', 
                                      'SpliceAI_pred_DS_AL':'SpliceAI_AL',
                                      'SpliceAI_pred_DS_DG': 'SpliceAI_DG',
                                      'SpliceAI_pred_DS_DL': 'SpliceAI_DL',
                                      'am_pathogenicity': 'am_score',
                                      'CADD_PHRED': 'cadd_score',
                                      'REVEL': 'revel_score'
                                     })

    mutpred_df = mutpred_df[['hg38_start', 'alt_allele', 'MutPred2']]
    mutpred_df['pos_id'] = mutpred_df['hg38_start'].astype(str) + ':' + mutpred_df['alt_allele']
    
    mutpred_df = mutpred_df[['pos_id', 'MutPred2']]
    vep_df = vep_df[['pos_id', 'SpliceAI_AG', 'SpliceAI_AL', 'SpliceAI_DG', 'SpliceAI_DL', 'max_SpliceAI', 'am_score', 'cadd_score', 'revel_score']]
    vep_df = pd.merge(vep_df, mutpred_df, on = 'pos_id', how = 'left')
    
    return vep_df

Functions to add median normalized depth data

In [ ]:
def read_input(file, cut_sites): #Reads input file containing coordinates for all exons
    
    input_params = pd.read_excel(file) #Reads input file

    #Loop that creates list of genomic coordinates for coding sequence
    i = 0
    cds_coords = [] #List to hold coding coordinates
    while i < len(input_params):
        start = input_params['start'][i] #Gets starting coordinate
        end = input_params['end'][i] #Gets end coordinate

        #Makes coding coordinates
        for j in range(start, end + 1):
            cds_coords.append(j)
        
        i += 1

    cut_sites = pd.read_excel(cut_sites)
    cut_sites.set_index('target', inplace = True)

    return cds_coords, cut_sites

In [ ]:
def process_depth(depth_path, raw_sge,coding_coords, target_coords, cut_df): #Processes all depth files in directory and annotates them
    
    file_path = Path(depth_path) #Creates path object for depth files 

    depth_files = sorted(list(file_path.glob('*.tsv'))) #Gets all depth files
    
    columns = ['region', 'offset', 'depth'] #Column names for renaming depth files
    
    target_coordinates = pd.read_excel(target_coords, sheet_name = 'targets') #Reads input with SGE target coordinates
    target_coordinates.set_index('target', inplace = True) #Sets target name to index

    all_dfs = [] #Empty list to hold processed dataframes

    #For loop iterates through all depth files and processes them
    for file in depth_files:
        df = pd.read_csv(file, sep = '\t') #Reads depth file
        df = df.set_axis(columns, axis = 1) #Renames columns 

        min_depth = df['depth'].min() #Gets minimum read count in file
        max_depth = df['depth'].max() #Gets maximum read count in file

        df['normdepth'] = df['depth'] / max_depth #Calculates normalized depth based on proportion of maximum read counts 

        full_region = df['region'][0] #Gets full SGE target
        region_start = int(re.findall(r':(\d+)-', full_region)[0]) #Gets starting coordinate for sequencing amplicon 

        df['pos'] = region_start + df['offset'] - 1 #Generates genomic coordinates for all regions based on offset column and starting coordinate

        file_str = str(file) #Sets file name to string data type
        region_rep = re.findall(r'/([^/]+)_D13\.depth\.tsv$', file_str)[0] #Gets region and replicate string

        region_rep_split = region_rep.split('_') #Splits string on '_'
        target = region_rep_split[0] + '_' + region_rep_split[1] #Gets target name
        exon_test = region_rep_split[1][1:-1] #Gets exon 

        region_start = target_coordinates.loc[target, 'end'] #Gets SGE target starting coordinate
        region_end = target_coordinates.loc[target, 'start'] #Gets SGE target end coordinate (end/start flipped due to antisense gene)
    
        region_coords = [] #List to hold coordinates in SGE target

        for k in range(region_start, region_end + 1): #Loop creates coordinates for SGE target
            region_coords.append(k)

        #Booleans to get name of exon
        if len(exon_test) > 0: #tests for all regions but X2
            exon = exon_test
        else: #Exception for X2
            exon = '2'
            
        full_rep = region_rep_split[2] #Gets replicate value

        #Boolean tests to assign replicate number
        if full_rep == 'R1R4' or full_rep == 'R1R2R3':
            rep = 'R1'
        elif full_rep == 'R2R5' or full_rep == 'R4R5R6':
            rep = 'R2'
        elif full_rep == 'R3R6' or full_rep == 'R7R8R9':
            rep = 'R3'

        cut_site = cut_df.loc[target, 'pos']
        
        #Sets columns with identifying information
        df['target'] = target
        df['exon'] = exon
        df['repl'] = rep
        df['day'] = 'D13'
        
        df = df.loc[df['pos'].isin(region_coords)] #Dataframe filtered for coordinates in SGE target edited region only

        df['cut_site_distance'] = -(df['pos'] - cut_site)
         
        all_dfs.append(df) #Dataframe appended to list
    
    final_df = pd.concat(all_dfs) #All dataframes concatenated
    final_df = final_df.loc[final_df['pos'].isin(coding_coords)] #Dataframes filtered for coding sequencing only 


    raw_sge['amino_acid'] = raw_sge['amino_acid_change'].transform(lambda x: x[0:-1]) #Creates amino acid column 
    raw_sge = raw_sge.loc[~(raw_sge['amino_acid'].isin(['--']))] #Drops columns without amino acid 
    annotation_df = raw_sge[['pos', 'amino_acid']] #Keeps position column for mergining and amino acid column for annotation 

    final_df = pd.merge(final_df, annotation_df, on = 'pos', how = 'left') #Depth and annotation_df merged to annotate with amino acids
   
    

    final_df['id'] = final_df['pos']  + final_df['depth']  + final_df['normdepth'] #Unique ID created for each datapoint 

    final_df = final_df.drop_duplicates(subset = 'id', keep = 'first') #Any duplicates dropped
    final_df = final_df.drop(columns = ['id']) #ID column dropped 

    return final_df    

In [ ]:
def process_read_depth(df): #Depth dataframe processed for visualization

    df['pos'] = df['pos'].astype(str) #Sets 'pos' column to string datatype
    df['target_id'] = df['target'] + ':' + df['pos'] #Builds a target ID column for target-based collapsing to median
    
    grouped = df.groupby('target_id') #Groups dataframe by target ID

    #Creates dataframe with annotated CDS positions
    cds_annotated = df.groupby('pos').agg({ #Grouping by position allows for accurate CDS pos. to be assigned
    'normdepth': 'median', 
    'target': 'first',
    'exon': 'first',
    'amino_acid': 'last'
    }).reset_index()

    
    cds_pos = [] #List to hold CDS position values

    for i in range(len(cds_annotated)): #Builds CDS position values
        cds_pos.append(i+1)

    cds_pos = cds_pos[::-1] #Reverses values due to negative sense gene

    
    
    cds_annotated['CDSpos'] = cds_pos #Adds CDS position column
    cds_annotated = cds_annotated[['pos', 'CDSpos']] #Drops unncessary column s
    
    #Collapses depth calculations to median based on shared target and position for all 3 replicates
    median_depth = grouped.agg({
        'normdepth': 'median',
        'pos': 'first',
        'target': 'first',
        'exon': 'first',
        'amino_acid': 'first',
        'cut_site_distance': 'first'
    }
                              )

    median_depth = pd.merge(median_depth, cds_annotated, on = 'pos', how = 'left') #Merges with CDS annotated dataframe with add CDS position
    median_depth = median_depth.rename(columns = {'normdepth': 'median_depth'}) #depth column renamed 
    
    median_depth['AApos'] = median_depth['amino_acid'].str[1:]
    #median_depth.to_excel('/Users/ivan/Desktop/test_excel_outputs/20250806_DepthOutput.xlsx', index = None)

    aa_grouped = median_depth.groupby('AApos')

    min_depth_aa_level = aa_grouped.agg({
        'median_depth': 'min',
        'target': 'first', 
        'exon': 'first',
        'amino_acid': 'first',
    }
                                          )
    min_depth_aa_level = min_depth_aa_level.reset_index(names = ['AApos'])
    
    return median_depth, min_depth_aa_level

In [ ]:
def main():

    #Reads SGE data
    snv_df, snv_thresholds = get_thresholds(snv_file, thresholds)
    del_df = class_dels(del_file, snv_thresholds)

    sge_df = pd.concat([snv_df, del_df]) #Final concatenated SNVs and Deletions dataframe

    #Reads and Merges RNA data
    rna_df,rna_thresh = process_rna(rna_scores)

    df = pd.merge(sge_df, rna_df, on = 'pos_id', how = 'left') #Merged with RNA scores and classifications


    #Processes and Merges ClinVar Data
    clinvar_snvs, clinvar_dels = read_clinvar(clinvar_snvs_file, clinvar_dels_file)
    clinvar_snvs = get_base_changes(clinvar_snvs)
    all_clinvar = pd.concat([clinvar_snvs, clinvar_dels])
    
    df = pd.merge(df, all_clinvar, on = 'pos_id', how = 'left') #df merged with ClinVar

    #Processes and Merges MAF Data
    gnomad_df = read_gnomAD(gnom_path)
    regeneron_df = read_regeneron(reg_path)

    df = pd.merge(df, gnomad_df, on = 'pos_id', how = 'left')
    df = pd.merge(df, regeneron_df, on = 'pos_id', how = 'left')

    #Merge in SNV counts
    counts_df = read_counts(snv_counts)

    #Merge PhyloP data
    phylop_df = pd.read_excel(phylop)
    phylop_df['pos'] = phylop_df['pos'].astype(int)
    phylop_df = phylop_df.drop_duplicates(subset = ['phyloP'])
    df = pd.merge(df, phylop_df, on = 'pos', how = 'left')

    #Adds VEP data
    vep_df = read_vep(vep_predictions, mutpred_prediction)

    df = pd.merge(df, vep_df, on = 'pos_id', how = 'left')

    #Calculates and adds normalized read depth for each position
    coding_coords, cut_coords = read_input(gsp_input_file, cut_sites)
    all_reps_depth = process_depth(read_depth_path, snv_df, coding_coords, target_coords, cut_coords)
    collapsed_depth, min_collapsed_depth_aa = process_read_depth(all_reps_depth)


    #Adds ATG data
    atg_df = pd.read_excel(atg_scores)

    
    threshold_df = pd.DataFrame({'min': [snv_thresholds[0]], 'max': [snv_thresholds[1]], 'rna': [rna_thresh]})

    #Adds edit rates
    edit_df = pd.read_csv(edit_rate, sep = '\t')

    #Adds orthogonal assays
    orthogonal_df = pd.read_excel(orthogonal_assays)

    dfs = {'scores': df,
           'snv_counts': counts_df,
           'edit_rates': edit_df,
           'thresholds': threshold_df,
           'cutsites': cut_coords,
           'median_pos_depth': collapsed_depth,
           'min_aa_depth': min_collapsed_depth_aa,
           'start_codon_scores': atg_df,
           'orthogonal_data': orthogonal_df
          }



    print('Writing File...')

    with pd.ExcelWriter('../Data/final_tables/BARD1_SGE_final_table.xlsx') as writer:
        for sheet_name, df in dfs.items():
            df.to_excel(writer, sheet_name=sheet_name, index=False)
  
    print('File Successfully Written')

    

In [ ]:
main()